# Imaging Methods in Granular & Complex Fluids

# Day 2: Intro to `scikit-image` and particle finding (1 of 2)

Karen Daniels, Distinguished Professor of Physics, NC State University kdaniel@ncsu.edu

Visiting Fulbright-Nehru Scholar,  IISc Civil Engineering, September 2023 to January 2024

## Learning objectives

* Use a Jupyter Notebook within anaconda
* Reinforce skills at loading and manipulating digital images
* Use `scikit-image` to 
     * modify an image to make it easier to find particles
     * find particle centroids using Hough transform
     * measure particle properties obtained through a basic form of image segementation (thresholding)


# Pre-class

* *Pre-class reading:* [Hough Transform Primer](https://www.scaler.com/topics/hough-transform-in-image-processing/)
* *Bring to class:* smartphone, laptop with anaconda with `scipy`, `matplotlib`, and `scikit-image` installed, photos from last week

# In-Class

## Testing anaconda installation

Check that you can load these packages without errors. If not, use the Ananconda GUI to find and install them, or else use the command line (`conda install skimage`)

In [ ]:
import numpy as np              # for scientific calculations
import matplotlib.pyplot as plt # for plotting
import skimage                  # for image-processing

## Getting started, with a low-quality image


Make sure that you have saved the `apples-oranges.jpg` image in the same subdirectory where you saved this Jupyter notebook, since this will be the simplest way for the notebooks to find the right file.

We're also going to turn on a useful feature of matplotlib: interactive plots. This will let you zoom in/out within plots to examine them more closely. Once you're done examining, simply click the "power" button up the upper right to turn off interactivity. 

If you don't like this feature, comment out the first two lines by putting a # in front of them. You can also use the command `plt.ioff()` will turn the interactivity off for future plots. 

In [ ]:
%matplotlib notebook
plt.ion()

img = skimage.io.imread('apples-oranges.jpg')
#img = io.imread('subdir/my-picture.jpg')   #use this if the picture is inside a subdirectory (Mac/Linux)
#img = io.imread('subdir\my-picture.jpg')   #use this if the picture is inside a subdirectory (Windows)
plt.imshow(img)

Our aims are to: 
* identify the individual circle-like objects within the image
* find the coordinate of their centers ("centroids")
* measure the sizes of the objects

To get started, use the scales (in pixels) at the edge of the picture to estimate the typical radius of the objects in the picture.

In [ ]:
# use the interactive zoom feature to decide a reasonable radius range for the objects
# type those min/max values here:

Rmin = 
Rmax = 

## Using `scikit-image` Hough transform to identify circle-like objects

You can read the [Documentation](https://scikit-image.org/docs/stable/auto_examples/edges/plot_circular_elliptical_hough_transform.html) for this package to find out more about the options; the code below is modified from that source.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# loading just the parts of the scikit-image package that we'll be using
from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte

# Load picture as unsigned 8-bit integers
img = skimage.io.imread('apples-oranges.jpg')
image = img_as_ubyte(img[:,:,2])  # just the blue channel, which is similar for both fruits
plt.imshow(image, cmap=plt.cm.gray)
plt.show()

The Hough transforms works on outlines of circles, not on filled-in disks, so we first need to use an edge-detection algorithm to find the sharpest edges in the picture. One common method is [Canny edge detection](https://en.wikipedia.org/wiki/Canny_edge_detector), and the [`skimage` documentation](https://scikit-image.org/docs/stable/auto_examples/edges/plot_canny.html) describes its implementation here. 

Use the interactive zoom to explore, and you should see a nice round outline of each fruit, but also some smaller features inside (that we're not interested in).

In [ ]:
# The Canny these settings might need to be changed depending on the properties of your image
# if you want to try other values"
#      make a copy of the original line
#      comment out the original line
#      play with the values in the copy (so that the original is safe)

edges = canny(image, sigma=3, low_threshold=10, high_threshold=50) 
plt.imshow(edges)
plt.colorbar()
plt.show()

The Hough algorithm looks for circle-like objects within some range $R_{min}$ to $R_{max}$, and provides a score for each object. The user has the choice to look at all or some of the objects found. 

Run this code and take a look and zoom around for various choices of $R$. You should be able to find the highest scores near where you expect the centers of the objects.

In [ ]:
# Detect all objects with a radii within a set range from R_min to R_max in s
# the longer the list, the long the algorithm takes 
Rmin = 220
Rmax = 280
Rstep = 5
hough_radii = np.arange(Rmin, Rmax, Rstep)
print('Trying radii: ', hough_radii)
hough_res = hough_circle(edges, hough_radii)

# look at the scores for a particular radius being sought
i=4
plt.imshow(hough_res[i,:,:])
plt.title('Hough score for R=' + str(hough_radii[i]))
plt.colorbar()
plt.show()

In the image above, use the interactive zoom to look at someplace where you know there should be a center. You should see higher scores (yellower) for a small number of pixels. Because the apples and oranges aren't very circular, there isn't a nice, clean central peak.



Switch which of the radii on the list you're plotting at the end: can you see that some work better than others for some of the fruits, depending on the size of the fruit? **What do you notice?** 

*... your thoughts here ... (double click the box to edit it)*

Next, we'll use the python algorithm that finds those peaks. You can change the value of `total_num_peaks` if you'd like. The original value I put was 20, but that's not a particularly well-considered value.

In [ ]:
# Select the centroids of the most prominent 20 circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                           total_num_peaks=20)
plt.imshow(image, cmap=plt.cm.gray)
plt.plot(cx, cy, 'rx')
plt.show()

You'll notice that some objects get detcted more than once (many red `x` inside some fruits), when they have portions of their outer edge which match different radii circles. And others got missed entirely. 

**What types of objects do you expect the Hough transform to work best/worst on?** 

*... your thoughts here...*

Next, we'll examine how well the Hough transform worked by drawing the circles it found.

In [ ]:
# Draw them as circles
from matplotlib.patches import Circle

fig, ax = plt.subplots()
ax.imshow(image, cmap=plt.cm.gray)

for center_y, center_x, radius in zip(cy, cx, radii):
    circle = Circle((center_x, center_y), radius, edgecolor='blue', facecolor="none", linewidth=3, alpha=0.5)
    ax.add_patch(circle)

ax.set_aspect('equal')
plt.show()

**Using the interactive zoom, what do you notice about the circles it found?**

*... your thoughts here...*

## An example with cleaner circles

We're going to repeat the above example for an image (your own, or the `PEdisks.png` provided in today's subdirectory; download a copy to your own computer) for which the objects are more nearly circular.  If you'd like to use your own image, select one with good contrast, no strong gradients, and the particles at least approximately circular. 

Whichever images you use, be sure to save them in the same directory as this Jupyter Notebook.

Suggestion: duplicate this next cell in order to examine your own image, and leave the example code pristine. The "Edit" menu has some selectable items that will do this task for you.

If you want to see the intermediate steps, feel free to uncomment the plotting code. 

In [ ]:
img = skimage.io.imread('PEdisks.png')
image = img_as_ubyte(img[:,:,2])  # just the blue channel
#image[image > 60] = 255
#plt.imshow(image, cmap=plt.cm.gray)
#plt.show()

edges = canny(image, sigma=3, low_threshold=10, high_threshold=50) 
#plt.imshow(edges)
#plt.colorbar()
#plt.show()

hough_radii = np.array([19,28])
# only putting in 2 possible radii, since I know the correct values
hough_res = hough_circle(edges, hough_radii)
# look at the scores for a particular radius being sought
#i=4
#plt.imshow(hough_res[i,:,:])
#plt.title('Hough score for R=' + str(hough_radii[i]))
#plt.colorbar()
#plt.show()
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=32)

fig, ax = plt.subplots()
ax.imshow(image, cmap=plt.cm.gray)
for center_y, center_x, radius in zip(cy, cx, radii):
    circle = Circle((center_x, center_y), radius, edgecolor='blue', facecolor="none", linewidth=3, alpha=0.5)
    ax.add_patch(circle)
ax.plot(cx, cy, 'rx')
ax.set_aspect('equal')
plt.show()

**Compared to the fruits example, what do you notice is the same/different?**

*... your thoughts here...*

## Exploring properties of objects within an image

`scikit-image` also has some built-inpackages for image-segmentation which do not require the objects to be circles. 

*Some of this Module's content is adapteed from this [online tutorial](https://scipy-lectures.org/packages/scikit-image/index.html), which has additional examples if you're interested in learning more.*

Documentation for the `measure` toolbox: https://scikit-image.org/docs/dev/api/skimage.measure.html

Documenation for the `morphology` toolbox: https://scikit-image.org/docs/stable/api/skimage.morphology.html

In [ ]:
from skimage import measure    # give ourselves a shortcut to some more tools

In [ ]:
# Find all regions above the mean
img = skimage.io.imread('PEdisks.png')  # or your own image if you prefer
img = img_as_ubyte(img[:,:,2])
threshold = img > 0.8*img.mean()
# Ask yourself: what does that line of code mean?

plt.imshow(threshold, cmap='gray')
plt.title('thresholded image')
plt.show()

In [ ]:
# Label each connected-component with a unique name (will be a unique color)
all_labels = measure.label(threshold)  
# Each region of the image "threshold" is saved into
# a new image named "all_labels" where the pixel has the 
# a value that is the label for that region

# more info about this funcion here:
# https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.label
# which also says a little bit about how it works

# we can see how this works by looking at the data inside the image
plt.imshow(all_labels,cmap='plasma')
# for example:
row = 120
col = 170
plt.plot(col,row, 'gx')
plt.colorbar()
plt.show()
print('x marks the region named', all_labels[row,col])

**To do:** move the row and col values around to see the different name inside different regions. You'll see that may particles are lumped into the same region.  Can out out how the color/number works, based on the colorbar? While in interactive mode, notice the displayed numbers in the lower right that correspond to the current mouse location.

*... space to write some notes ...*

We'll modify the code above to set a higher threshold, and then "erode" the circles slightly, to give them some separation. Better? You can play with the threshold and repeat the erosion more times, if you'd like.

In [ ]:
img = skimage.io.imread('PEdisks.png')  # or your own image if you prefer
img = img_as_ubyte(img[:,:,2])
threshold = img > 0.9*img.mean()
threshold = skimage.morphology.erosion(threshold)
threshold = skimage.morphology.erosion(threshold)
# if you curious how python packages work: not that I didn't give a shortcut name to morphology
# this means that we have to type out the full name in order to use it

plt.imshow(threshold, cmap='gray')
plt.title('thresholded image')
plt.show()

**What are the pros/cons between this method and the Hough method?** 

*...space for your ideas here ...*

In [ ]:
# trying labeling again, see improvement?
all_labels = measure.label(threshold)
plt.imshow(all_labels,cmap='plasma')
plt.colorbar()
plt.show()

# if you want to try morphological functions, read the docs and you can clean your regions up even more

Next, we'll measure some of the properties of these regions we found.

In [ ]:
regions = measure.regionprops(all_labels)

print('Found', len(regions), 'regions')

i=5
print('Examining region #', i)
print('Center coordinates', regions[i].centroid)


# can you 'Edit this line' to figure out what each one does?

ecc = [prop.eccentricity for prop in regions]
print('Edit this line:', ecc[i])

rowscols = regions[i].coords
print('Edit this line:', rowscols[i,0], rowscols[i,1], img[rowscols[i,0],rowscols[i,1]])

areas = [prop.area for prop in regions]
print('Edit this line:', areas[24])
count = 0
for i in range(len(regions)):
    if areas[i] > 10:
        count += 1
print('Edit this line:', count)


In [ ]:
# more space for you to play around with the code ... 
# try different types of images from the shared images directory, et. 

# Post-class

## Every week

If you and your group worked together on a single notebook, don't forget to email each other a copy of the final version.

Please feel free to complete any parts you didn't finish, and ask questions at the start of the next class.